In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

In [2]:
# 데이터 불러오기
Keyword = pd.read_csv('./Keyword.csv')
Keyword

,0
0,서울특별시 강남구 테헤란로52길 21 탐라정성돈카츠(탐정돈)
1,서울특별시 강남구 도산대로49길 10-3 로지
2,서울특별시 강남구 언주로98길 9 재패니즈 다이닝 안심
3,서울특별시 강남구 테헤란로39길 68 돈까스집이야 강남점
4,서울특별시 강남구 테헤란로39길 68 덮밥집이야 강남점
...,...
912,서울특별시 강남구 역삼로 115 (주)캘리스코 사보텐 아워홈빌딩점
913,서울특별시 강남구 봉은사로29길 35 만고쿠 딜리버리
914,서울특별시 강남구 압구정로 343 이치에 투고
915,서울특별시 강남구 강남대로84길 15 오이시포켓 강남역점


In [3]:
# 창이 나오지 않게 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'chromedriver.exe' # 크롬드라이버 경로 설정

source_url = "https://map.kakao.com/" # 리뷰주소 크롤링을 해 올 주소 설정

page_urls = [] #리뷰주소 크롤링 결과를 담을 리스트

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for k in tqdm_notebook(Keyword['0']):
    try:
        driver.get(source_url)

        # 검색
        search_bar = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')
        search_bar.send_keys(k)
        searchbutton = driver.find_element(By.CSS_SELECTOR, "#search\.keyword\.submit")
        driver.execute_script('arguments[0].click();', searchbutton)

        driver.implicitly_wait(10)

        # 상세보기에 링크된 리뷰주소 가져오기
        moreview = driver.find_element(By.CLASS_NAME, 'moreview')
        page_url = moreview.get_attribute("href")
        page_urls.append(page_url)
    except Exception as e:
        print(e)
        pass

driver.quit()

page_urls[:5]

  0%|          | 0/917 [00:00<?, ?it/s]

Message: no such element: Unable to locate element: {"method":"css selector","selector":".moreview"}
  (Session info: headless chrome=113.0.5672.126)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".moreview"}
  (Session info: headless chrome=113.0.5672.126)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".moreview"}
  (Session info: headless chrome=113.0.5672.126)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".moreview"}
  (Session info: headless chrome=113.0.5672.126)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".moreview"}
  (Session info: headless chrome=113.0.5672.126)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".moreview"}
  (Session info: headless chrome=113.0.5672.126)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".more

['https://place.map.kakao.com/11598105',
 'https://place.map.kakao.com/27364240',
 'https://place.map.kakao.com/87087038',
 'https://place.map.kakao.com/1182248276',
 'https://place.map.kakao.com/1475948300']

In [4]:
# 이미 리뷰주소가 있을 경우 사용
'''page_urls = []

for i in names_df.리뷰주소:
    page_urls.append(i)

options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'C:/Users/USER/TIL/chromedriver' '''

"page_urls = []\n\nfor i in names_df.리뷰주소:\n    page_urls.append(i)\n\noptions = webdriver.ChromeOptions()\noptions.add_argument('headless')\n\npath = 'C:/Users/USER/TIL/chromedriver' "

In [5]:
columns = ['name', 'score', 'review']
review_df = pd.DataFrame(columns=columns)

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for page_url in tqdm_notebook(page_urls):
    try:
        driver.get(page_url)
        driver.implicitly_wait(10)

        # 식당명, 평점, 리뷰 가져오기
        name = driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > h2')
        name = name.text
        star = driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_evaluation > div.evaluation_review > ul > li:nth-child(1) > div.star_info > div > span > span')
        star = ''.join([i for i in star.get_attribute("style") if i.isdigit()])
        review = driver.find_elements(By.CLASS_NAME, 'txt_comment')
        for i in review:
            review_df.loc[len(review_df)] = [name, star, i.text]

    except Exception as e:
        # 리뷰가 없거나 비공개인 경우
        print(e)
        pass
    
driver.quit()

review_df.head()

  0%|          | 0/556 [00:00<?, ?it/s]

Message: no such element: Unable to locate element: {"method":"css selector","selector":"#mArticle > div.cont_evaluation > div.evaluation_review > ul > li:nth-child(1) > div.star_info > div > span > span"}
  (Session info: headless chrome=113.0.5672.126)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"#mArticle > div.cont_evaluation > div.evaluation_review > ul > li:nth-child(1) > div.star_info > div > span > span"}
  (Session info: headless chrome=113.0.5672.126)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"#mArticle > div.cont_evaluation > div.evaluation_review > ul > li:nth-child(1) > div.star_info > div > span > span"}
  (Session info: headless chrome=113.0.5672.126)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"#mArticle > div.cont_evaluation > div.evaluation_review > ul > li:nth-child(1) > div.star_info > div > span > span"}
  (Session info: headless 

,name,score,review
0,로지,100,모둠 사시미에 줄서 봅니다.
1,로지,100,
2,로지,100,너무 숙성된(?) 사시미...친절하지만 맛은 그다지.
3,재패니즈다이닝안심,80,맛있습니다 잘먹었습니다\n또 갈거 같은데 뭔가 모르게\n와~~하기에는 좀\n아쉬움이...
4,재패니즈다이닝안심,80,역대급 맛집이에요


In [6]:
# 결측값 제거
review_df = review_df[review_df['review']!='']
review_df

,name,score,review
0,로지,100,모둠 사시미에 줄서 봅니다.
2,로지,100,너무 숙성된(?) 사시미...친절하지만 맛은 그다지.
3,재패니즈다이닝안심,80,맛있습니다 잘먹었습니다\n또 갈거 같은데 뭔가 모르게\n와~~하기에는 좀\n아쉬움이...
4,재패니즈다이닝안심,80,역대급 맛집이에요
5,재패니즈다이닝안심,80,사시미랑 구이랑 모두 다 너무 훌륭합니다 !
...,...,...,...
1336,카츠하나,100,잘함
1337,스시아오마츠,100,최근에 먹은곳중 가장 가격이상의 가치를 만드는곳
1338,스시아오마츠,100,근래에 방문한 곳 중에 가장 만족스러워요 😋
1339,스시아오마츠,100,샤리가 고슬고슬해서 맛있었어요\n디너였는데 양이 많아서 거의 2-3시간 먹은 것 같...


In [7]:
review_df.to_csv("./review_data.csv", index=False)